In [4]:
import numpy as np
import matplotlib
import pandas as pd

dataset = pd.read_csv('/Users/eddie/Downloads/lake_erie_habs_W4_6_8_9_12_13_16_2013-2020.xlsx - HAB_data.csv')

labels = dataset.iloc[:,16:].values

vals = pd.DataFrame(dataset.iloc[:,4:15].values)

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(vals,labels,test_size=0.2,random_state=0)
X_train = np.array(X_train)


In [5]:
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.ensemble import RandomForestRegressor

In [6]:
# Initialize MLP model
def create_mlp():
    mlp_model = Sequential([
        Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(1)  # Output layer for regression task
    ])

    # Compile the model
    mlp_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), loss='mae')

    # Print model summary
    mlp_model.summary()

    # Train the model
    history = mlp_model.fit(X_train, Y_train, epochs=300, batch_size=32, validation_split=0.2)

    # Evaluate model performance
    Y_pred = mlp_model.predict(X_test)
    return mlp_model


In [7]:
def create_rf():
    # Initialize Random Forest regressor
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

    # Train the model
    rf_model.fit(X_train, Y_train)
    return rf_model


In [8]:
from tensorflow.keras.layers import concatenate

In [9]:
rf = create_rf()
mlp = create_mlp()
mlpPredict = mlp.predict(X_train)
rfPredict = rf.predict(X_train)
mlpPredict.resize(671,1)
rfPredict.resize(671,1)
trainData = concatenate([mlpPredict, rfPredict])

c:\Users\eddie\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:1351: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3393 (13.25 KB)
Trainable params: 3393 (13.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/300

17/17 [==============================] - 1s 16ms/step - loss: 27.6225 - val_loss: 21.2547
Epoch 2/300
17/17 [==============================]

In [20]:
from xgboost import dask as dxgb
from dask import array as da



In [21]:
regressor = dxgb.DaskXGBRegressor(verbosity=1, n_estimators=2)
regressor.set_params(tree_method="hist")
regressor.fit(trainData, Y_train)

Windows is not officially supported for dask/xgboost, contribution are welcomed.


TypeError: unhashable type: 'list'

In [ ]:
mlpPredictTest = mlp.predict(X_test)
rfPredictTest = rf.predict(X_test)
rfPredictTest.resize(168,1)
testData = concatenate([mlpPredictTest,rfPredictTest])

In [ ]:
regressor.predict(testData,Y_test)